In [42]:
import pandas as pd 
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
res_roberta_train_df = pd.read_json("./results/predictions-train-transformers-roberta-roberta-base-batch-32-epochs-1-learn-2e-05-warmup-0.06.json").head(n=10)
res_roberta_dev_df = pd.read_json("./results/predictions-dev-transformers-roberta-roberta-base-batch-32-epochs-1-learn-2e-05-warmup-0.06.json")
res_robert_test_df = pd.read_json("./results/predictions-test-transformers-roberta-roberta-base-batch-32-epochs-1-learn-2e-05-warmup-0.06.json")

labels_train_df = pd.read_csv("./../data/labels_train.csv")
arg_train_df = pd.read_csv("./../data/arguments_train.csv")
kp_train_df = pd.read_csv("./../data/key_points_train.csv")

In [43]:
def load_predictions(predictions_file: Path) -> pd.DataFrame:
    """
    Load data frame with argument key point matches and scores.
    """
    args: List[str] = []
    kps: List[str] = []
    scores: List[float] = []
    with predictions_file.open("r") as file_in:
        predictions = json.load(file_in)
    for arg_id, kp_scores in predictions.items():
        for kp_id, score in kp_scores.items():
            args.append(arg_id)
            kps.append(kp_id)
            scores.append(score)
    print(f"Loaded {len(args)} predictions "
          f"for {len(predictions.items())} arguments.")
    return pd.DataFrame({
        "arg_id": args,
        "key_point_id": kps,
        "score": scores
    })

In [44]:
pred_roberta_train_df = load_predictions(Path('./results/predictions-train-transformers-roberta-roberta-base-batch-32-epochs-1-learn-2e-05-warmup-0.06.json'))

Loaded 24454 predictions for 5583 arguments.


In [48]:
roberta_train_df = labels_train_df.merge(arg_train_df, on='arg_id').merge(kp_train_df, on='key_point_id')
roberta_train_df = roberta_train_df[['arg_id','argument','key_point_id','key_point']].merge(pred_roberta_train_df, on=['arg_id','key_point_id'])
roberta_train_df.set_index(['arg_id','key'])

roberta_train_df




,arg_id,argument,key_point_id,key_point,score
0,arg_0_0,`people reach their limit when it comes to the...,kp_0_0,Assisted suicide gives dignity to the person t...,0.043823
1,arg_0_1,A patient should be able to decide when they h...,kp_0_0,Assisted suicide gives dignity to the person t...,-0.002674
2,arg_0_2,a person has the right to end their suffering ...,kp_0_0,Assisted suicide gives dignity to the person t...,0.054443
3,arg_0_4,a person should have the right to be able to c...,kp_0_0,Assisted suicide gives dignity to the person t...,0.032013
4,arg_0_5,a person should have the right to die on their...,kp_0_0,Assisted suicide gives dignity to the person t...,0.033020
...,...,...,...,...,...
20630,arg_27_90,we shouldn't subsidize vocational education. m...,kp_27_3,subsidizing vocational education is expensive,0.014900
20631,arg_27_91,we shouldn't support vocational education beca...,kp_27_3,subsidizing vocational education is expensive,-0.000334
20632,arg_27_92,we shouldn't support vocational education beca...,kp_27_3,subsidizing vocational education is expensive,0.882324
20633,arg_27_93,when people get things free or reduced i think...,kp_27_3,subsidizing vocational education is expensive,0.007435


In [46]:
# plt.figure()

roberta_train_match_label_df.head(n=200).plot(kind="density")


LinAlgError: singular matrix